In [106]:
import pandas as pd

from main import ParseReport, fuzz

report = ParseReport(file="result/TEST - Magellan - Cloud Example and Explanation.json", json_file=True, save_json=False)
report.get_lien()

[{'Lien Type:': 'Federal Tax Lien',
  'Filed Against:': None,
  'Amount:': '$87,052.04',
  'Recorded Date:': '01/10/2020',
  'Recording Information:': None,
  'Comment:': None},
 {'Lien Type:': 'Federal Tax Lien',
  'Filed Against:': None,
  'Amount:': '$35,986.01',
  'Recorded Date:': '03/08/2010',
  'Recording Information:': None,
  'Comment:': None}]

In [107]:
report.words.head()

,page_idx,dimensions,orientation.value,orientation.confidence,language.value,language.confidence,block_idx,block_geometry,artefacts,line_idx,line_geometry,word_idx,value,confidence,x1,y1,x2,y2
index,,,,,,,,,,,,,,,,,,
0,0,"[1684, 1190]",None,None,None,None,0,"[[0.4365234375, 0.0556640625], [0.9267578125, ...",[],0,"[[0.4365234375, 0.0556640625], [0.9267578125, ...",0,Southern,0.999931,0.436523,0.056641,0.545898,0.074219
1,0,"[1684, 1190]",None,None,None,None,0,"[[0.4365234375, 0.0556640625], [0.9267578125, ...",[],0,"[[0.4365234375, 0.0556640625], [0.9267578125, ...",1,Indiana,0.962854,0.552734,0.056641,0.643555,0.074219
2,0,"[1684, 1190]",None,None,None,None,0,"[[0.4365234375, 0.0556640625], [0.9267578125, ...",[],0,"[[0.4365234375, 0.0556640625], [0.9267578125, ...",2,Abstract,0.979812,0.649414,0.058594,0.750977,0.073242
3,0,"[1684, 1190]",None,None,None,None,0,"[[0.4365234375, 0.0556640625], [0.9267578125, ...",[],0,"[[0.4365234375, 0.0556640625], [0.9267578125, ...",3,"Company,",0.980987,0.756836,0.057617,0.879883,0.078125
4,0,"[1684, 1190]",None,None,None,None,0,"[[0.4365234375, 0.0556640625], [0.9267578125, ...",[],0,"[[0.4365234375, 0.0556640625], [0.9267578125, ...",4,Inc,0.987708,0.886719,0.055664,0.926758,0.075195


In [108]:
context = report.words
word1, word2 = "Notice", "Lien"
line = context[context['value'].str.contains(word1) | context['value'].str.contains(word2)][["page_idx", "block_idx", "line_idx", "x1", "y1", "x2", "y2", "value"]]
        
line['paired']=  line.value + " " + line.value.shift(-1)
line['space']=  line.x1.shift(-1) - line.x2
line['align']=  (abs(line.y1.shift(-1) - line.y1) + (line.y2.shift(-1) - line.y2))/2

line["score"] = line["paired"].apply(lambda x: 100 - fuzz.ratio(str(x), str(word1) + " " + str(word2)))
line = line[line.score < 10]
line = line.sort_values(by=['score', 'space', "align"])
# line = line.iloc[0][["page_idx", "block_idx", "line_idx", "paired"]]
# line = context[(context['page_idx'] == line.page_idx.item()) & (context['block_idx'] == line.block_idx.item()) & (context['line_idx'] == line.line_idx.item())]["value"].values
line

,page_idx,block_idx,line_idx,x1,y1,x2,y2,value,paired,space,align,score
index,,,,,,,,,,,,
2374,13,222,517,0.424805,0.083984,0.485352,0.098633,Notice,Notice Lien,0.133789,0.000000,0
2021,12,187,427,0.420898,0.089844,0.479492,0.101562,Notice,Notice Lien,0.134766,0.001465,0
3020,15,269,681,0.250977,0.227539,0.312500,0.238281,Notices,"Notices Lien,",0.191406,0.092285,8


In [109]:
context = report.words
word1, word2 = "Notice", "Lien"
line = context[context['value'].str.contains(word1) | context['value'].str.contains(word2)][["page_idx", "block_idx", "line_idx", "x1", "y1", "x2", "y2", "value"]]

line['paired']=  line.value + " " + line.value.shift(-1)
line['space']=  line.x1.shift(-1) - line.x2
line['align']=  (abs(line.y1.shift(-1) - line.y1) + (line.y2.shift(-1) - line.y2))/2

line["score"] = line["paired"].apply(lambda x: 100 - fuzz.ratio(str(x), str(word1) + " " + str(word2)))
line = line[line.score < 10]
line = line.sort_values(by=['score', 'space', "align"])
result = line.iloc[0][["page_idx", "block_idx", "line_idx", "paired"]]
result = context[(context['page_idx'] == result.page_idx.item()) & (context['block_idx'] == result.block_idx.item()) & (context['line_idx'] == result.line_idx.item())]["value"].values
result


array(['Notice', 'of', 'Federal', 'Tax', 'Lien'], dtype=object)

In [110]:
result = line.iloc[0][["page_idx", "block_idx", "line_idx", "paired"]]
result = context[(context['page_idx'] == result.page_idx.item()) & (context['block_idx'] == result.block_idx.item()) & (context['line_idx'] == result.line_idx.item())]["value"].values
result

array(['Notice', 'of', 'Federal', 'Tax', 'Lien'], dtype=object)

In [111]:
word1, word2 = "Kind", "Tax"
row = line.iloc[0][["page_idx", "block_idx", "line_idx", "paired"]]
c = report.words[(report.words.page_idx == row.page_idx)]
column = c[c['value'].str.contains(word1) | c['value'].str.contains(word2)][["page_idx", "block_idx", "line_idx", "x1", "y1", "x2", "y2", "value"]]
column['paired']=  column.value + " " + column.value.shift(-1)
column['space']=  abs(column.x1.shift(-1) - column.x2)
column['align']=  (abs(column.y1.shift(-1) - column.y1) + (column.y2.shift(-1) - column.y2))/2

column['x12']=  column.x1.shift(-1)
column['y12']=  column.y1.shift(-1)
column['x22']=  column.x2.shift(-1)
column['y22']=  column.y2.shift(-1)

column["score"] = column["paired"].apply(lambda x: 100 - fuzz.ratio(str(x), str(word1) + " " + str(word2)))
column = column[column.score < 10]
column = column.sort_values(by=['score', 'space', "align"])
column

,page_idx,block_idx,line_idx,x1,y1,x2,y2,value,paired,space,align,x12,y12,x22,y22,score
index,,,,,,,,,,,,,,,,
2536,13,231,540,0.170898,0.344727,0.204102,0.356445,Kind,Kind Tax,0.016602,0.0,0.220703,0.344727,0.249023,0.356445,0


In [112]:
c[(c.block_idx==231)][["value", "line_idx"]]

,value,line_idx
index,,
2533,in,539
2534,IRC,539
2535,6325(a).,539
2536,Kind,540
2537,of,540
2538,Tax,540
2539,(a),541


In [113]:
right = 0.00
left = 0.00
column_data = c[(c.x1 >= column.x1.item() - left) & (c.x2 <= column.x22.item() + right) & (c.y1 >= column.y2.item())]
column_data['hd']=  abs(column_data.y1.shift(-1) - column.y2.item())
column_data = column_data.sort_values(by=['hd'])
column_data['bid']=  abs(column_data.block_idx.shift(-1) - column_data.block_idx)
column_data['lid']=  abs(column_data.line_idx.shift(-1) - column_data.line_idx)
column_data['h2d']=  abs(column_data.hd.shift(-1) - column_data.hd)
column_data = column_data.sort_values(by=['hd', 'h2d', 'bid', "lid"])
column_data = column_data.reset_index(drop=True)
column_data['hd2']=  column_data.y1.shift(-1) - column_data.y1
values = []

for i, k in column_data.iterrows():
    print(k.value, k.bid, k.lid, k.h2d)
    print(k.bid - column_data.iloc[i + 1].bid.item() > 0)
    print(k.lid - column_data.iloc[i + 1].lid.item() > 0)
    print(column_data.iloc[i + 1].h2d.item() > 0.12)
    print(i)

    if k.hd > 0.2 or column_data.iloc[i + 1].h2d.item() > 0.2:
    # print(column_data.bid - column_data.iloc[i + 1].bid.item())
    # print(k.bid - column_data.iloc[i + 1].bid.item())
        break
    else:
        values.append(k.value)

# values
column_data[["value", "block_idx", "line_idx", "hd2", 'hd', 'h2d', 'bid', "lid"]]

1040 0.0 1.0 0.0107421875
False
False
False
0
1040 0.0 1.0 0.0107421875
False
False
False
1
1040 0.0 1.0 0.01171875
False
False
False
2
1040 0.0 1.0 0.0107421875
False
False
False
3
1040 0.0 1.0 0.0107421875
False
False
False
4
1040 0.0 1.0 0.1162109375
False
False
False
5
1040 1.0 1.0 0.0703125
False
False
False
6
of 4.0 7.0 0.0791015625
False
False
False
7


,value,block_idx,line_idx,hd2,hd,h2d,bid,lid
0,1040,237,559,0.010742,0.023438,0.010742,0.0,1.0
1,1040,237,560,0.010742,0.034180,0.010742,0.0,1.0
2,1040,237,561,0.010742,0.044922,0.011719,0.0,1.0
3,1040,237,562,0.011719,0.056641,0.010742,0.0,1.0
4,1040,237,563,0.010742,0.067383,0.010742,0.0,1.0
5,1040,237,564,0.010742,0.078125,0.116211,0.0,1.0
6,1040,237,565,0.116211,0.194336,0.070312,1.0,1.0
7,of,238,566,0.070312,0.264648,0.079102,4.0,7.0
8,notice,242,573,0.079102,0.343750,0.012695,6.0,13.0
9,G.J.,248,586,0.012695,0.356445,0.009766,0.0,1.0


In [114]:
right = 0.00
left = 0.00
height = 0.21
column_data = c[(c.x1 >= column.x1.item() - left) & (c.x2 <= column.x22.item() + right) & (c.y1 - 0.01 >= column.y2.item())]
column_data['hd'] = abs(column_data.y1.shift(-1) - column.y2.item())
column_data = column_data.sort_values(by=['hd'])
column_data['bid'] = abs(column_data.block_idx.shift(-1) - column_data.block_idx)
column_data['lid'] = abs(column_data.line_idx.shift(-1) - column_data.line_idx)
column_data['h2d'] = abs(column_data.hd.shift(-1) - column_data.hd)
column_data = column_data.sort_values(by=['hd', 'h2d', 'bid', "lid"])
column_data = column_data.reset_index(drop=True)
values = []
for i, k in column_data.iterrows():
    if row.block_idx == k.block_idx:
        continue
    # elif k.bid > 1 or k.lid > block_d or column_data.iloc[i + line_d].hd.item() > height:
    #     break
    if k.hd > height or column_data.iloc[i + 1].h2d.item() > height:
        break
    else:
        values.append(k)

values

[page_idx                                                                 13
 dimensions                                                     [1684, 1190]
 orientation.value                                                      None
 orientation.confidence                                                 None
 language.value                                                         None
 language.confidence                                                    None
 block_idx                                                               237
 block_geometry            [[0.193359375, 0.3681640625], [0.701171875, 0....
 artefacts                                                                []
 line_idx                                                                559
 line_geometry             [[0.193359375, 0.3681640625], [0.701171875, 0....
 word_idx                                                               2565
 value                                                                  1040

In [115]:
first_col = pd.DataFrame(values)
first_col

,page_idx,dimensions,orientation.value,orientation.confidence,language.value,language.confidence,block_idx,block_geometry,artefacts,line_idx,...,value,confidence,x1,y1,x2,y2,hd,bid,lid,h2d
0,13,"[1684, 1190]",None,None,None,None,237,"[[0.193359375, 0.3681640625], [0.701171875, 0....",[],559,...,1040,0.996726,0.193359,0.369141,0.233398,0.380859,0.023438,0.0,1.0,0.010742
1,13,"[1684, 1190]",None,None,None,None,237,"[[0.193359375, 0.3681640625], [0.701171875, 0....",[],560,...,1040,0.960193,0.193359,0.379883,0.233398,0.391602,0.034180,0.0,1.0,0.010742
2,13,"[1684, 1190]",None,None,None,None,237,"[[0.193359375, 0.3681640625], [0.701171875, 0....",[],561,...,1040,0.998010,0.193359,0.390625,0.233398,0.402344,0.044922,0.0,1.0,0.011719
3,13,"[1684, 1190]",None,None,None,None,237,"[[0.193359375, 0.3681640625], [0.701171875, 0....",[],562,...,1040,0.997673,0.193359,0.401367,0.233398,0.413086,0.056641,0.0,1.0,0.010742
4,13,"[1684, 1190]",None,None,None,None,237,"[[0.193359375, 0.3681640625], [0.701171875, 0....",[],563,...,1040,0.958626,0.193359,0.413086,0.233398,0.424805,0.067383,0.0,1.0,0.010742
5,13,"[1684, 1190]",None,None,None,None,237,"[[0.193359375, 0.3681640625], [0.701171875, 0....",[],564,...,1040,0.742312,0.193359,0.423828,0.233398,0.435547,0.078125,0.0,1.0,0.116211
6,13,"[1684, 1190]",None,None,None,None,237,"[[0.193359375, 0.3681640625], [0.701171875, 0....",[],565,...,1040,0.998738,0.194336,0.434570,0.233398,0.446289,0.194336,1.0,1.0,0.070312


In [116]:
for k, v in first_col.iterrows():
    row_values = list(c[c.line_idx == v.line_idx].value.values)
    line_values = c[(c.y1 >= c[c.line_idx == v.line_idx].y1.min()) & (c.y2 <= c[c.line_idx == v.line_idx].y2.max())].value.values
    row_values.append(line_values[0])
    print(row_values)
    

['1040', '12/31/2010', 'XXX-XX-5442', '12/10/2018', '01/09/2029', '5474.28']
['1040', '12/31/2011', 'XXX-XX-5442', '12/17/2018', '01/16/2029', '13622.93']
['1040', '12/31/2012', 'XXX-XX-5442', '12/17/2018', '01/16/2029', '13180.27']
['1040', '12/31/2013', 'XXX-XX-5442', '12/17/2018', '01/16/2029', '11788.82']
['1040', '12/31/2014', 'XXX-XX-5442', '12/17/2018', '01/16/2029', '10573.17']
['1040', '12/31/2015', 'XXX-XX-5442', '10/29/2018', '11/28/2028', '18465.47']
['1040', '12/31/2016', 'XXX-XX-5442', '10/29/2018', '11/28/2028', '13947.10']


In [117]:
context[(context.line_idx < 478) & (context.line_idx > 473)]

,page_idx,dimensions,orientation.value,orientation.confidence,language.value,language.confidence,block_idx,block_geometry,artefacts,line_idx,line_geometry,word_idx,value,confidence,x1,y1,x2,y2
index,,,,,,,,,,,,,,,,,,
2235,12,"[1684, 1190]",None,None,None,None,203,"[[0.1904296875, 0.3720703125], [0.693359375, 0...",[],474,"[[0.1904296875, 0.3828125], [0.693359375, 0.39...",2235,1040,0.979255,0.190430,0.383789,0.230469,0.395508
2236,12,"[1684, 1190]",None,None,None,None,203,"[[0.1904296875, 0.3720703125], [0.693359375, 0...",[],474,"[[0.1904296875, 0.3828125], [0.693359375, 0.39...",2236,12/31/2008,0.998142,0.254883,0.382812,0.350586,0.396484
2237,12,"[1684, 1190]",None,None,None,None,203,"[[0.1904296875, 0.3720703125], [0.693359375, 0...",[],474,"[[0.1904296875, 0.3828125], [0.693359375, 0.39...",2237,XXX-XX-5442,0.928731,0.367188,0.384766,0.470703,0.395508
2238,12,"[1684, 1190]",None,None,None,None,203,"[[0.1904296875, 0.3720703125], [0.693359375, 0...",[],474,"[[0.1904296875, 0.3828125], [0.693359375, 0.39...",2238,01/11/2010,0.681623,0.486328,0.382812,0.582031,0.396484
2239,12,"[1684, 1190]",None,None,None,None,203,"[[0.1904296875, 0.3720703125], [0.693359375, 0...",[],474,"[[0.1904296875, 0.3828125], [0.693359375, 0.39...",2239,02/10/2020,0.992605,0.598633,0.382812,0.693359,0.396484
2240,12,"[1684, 1190]",None,None,None,None,204,"[[0.1630859375, 0.5537109375], [0.2509765625, ...",[],475,"[[0.1630859375, 0.5537109375], [0.2509765625, ...",2240,Place,0.980629,0.163086,0.554688,0.199219,0.566406
2241,12,"[1684, 1190]",None,None,None,None,204,"[[0.1630859375, 0.5537109375], [0.2509765625, ...",[],475,"[[0.1630859375, 0.5537109375], [0.2509765625, ...",2241,of,0.999805,0.198242,0.553711,0.216797,0.566406
2242,12,"[1684, 1190]",None,None,None,None,204,"[[0.1630859375, 0.5537109375], [0.2509765625, ...",[],475,"[[0.1630859375, 0.5537109375], [0.2509765625, ...",2242,Filing,0.998490,0.216797,0.555664,0.250977,0.567383
2243,12,"[1684, 1190]",None,None,None,None,205,"[[0.2939453125, 0.5654296875], [0.4833984375, ...",[],476,"[[0.294921875, 0.5654296875], [0.4375, 0.57617...",2243,COUNTY,0.885574,0.294922,0.565430,0.353516,0.576172
